In [1]:
import numpy as np
import torch # PyTorch package
import torchvision.transforms as transforms # transform data
from torchvision import transforms, datasets
from pytorchtools import EarlyStopping
import time
from sklearn.metrics import classification_report
import splitfolders

#importing functions and parameters files
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.use("Agg")
import warnings
warnings.filterwarnings('ignore')

from functions import model, lossFn, opt
from params import bs, num_epochs, device, patience

c:\Users\dansh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Let's split the training folder into training and validation folders.

In [2]:
#only run one time
#splitfolders.ratio("images/training", output="training-validation",
#    seed=1337, ratio=(.8, .2), group_prefix=None, move=False) #

In [27]:
transform = transforms.Compose([
    transforms.CenterCrop(1080),
    transforms.Resize(224),
    transforms.ToTensor()])

In [28]:
#loading the data
train_data = datasets.ImageFolder('training-validation/train', transform = transform)
val_data = datasets.ImageFolder('training-validation/val', transform = transform)
test_data = datasets.ImageFolder('images/testing', transform = transform)
print(train_data.classes)

['flip', 'notflip']


In [5]:
# initialize the train, validation, and test data loaders
trainloader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = bs)
valloader = torch.utils.data.DataLoader(val_data, batch_size = bs)
testloader = torch.utils.data.DataLoader(test_data, batch_size = 1)

In [6]:
# calculate steps per epoch for training and validation set
trainSteps = len(trainloader.dataset) // bs
valSteps = len(valloader.dataset) // bs

In [7]:
inputs, classes = next(iter(trainloader))
print(f'Batch size: {inputs.shape[0]}')
print(f'Number of color channels: {inputs.shape[1]}')
print(f'Image size: {inputs.shape[2]} x {inputs.shape[3]}')

Batch size: 16
Number of color channels: 3
Image size: 224 x 224


In [8]:
# visualizing the images
plt.figure(figsize=(20, 10))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(inputs[i].permute(1, 2, 0))
    plt.title([train_data.classes[classes[i]]])

In [9]:
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()

[INFO] training the network...


In [10]:
#Train the model using early stopping
def train_model(model, bs, patience, num_epochs):

	# to store training history
	train_loss = []
	train_acc = []
	val_loss = []
	val_acc = []

	# initialize the early_stopping object
	early_stopping = EarlyStopping(patience=patience, verbose=True)

	#training loop
	# loop over our epochs
	for e in range(0, num_epochs):
		# set the model in training mode
		model.train()
		# initialize the total training and validation loss
		totalTrainLoss = 0
		totalValLoss = 0
		# initialize the number of correct predictions in the training
		# and validation step
		trainCorrect = 0
		valCorrect = 0
		# loop over the training set
		for (x, y) in trainloader:
			# send the input to the device
			(x, y) = (x.to(device), y.to(device))
			# clear the gradients of all optimized variables
			opt.zero_grad()
			# perform a forward pass and calculate the training loss
			pred = model(x)
			loss = lossFn(pred, y.type(torch.float32).unsqueeze(1))
			# zero out the gradients, perform the backpropagation step,
			# and update the weights
			loss.backward()
			opt.step()
			# add the loss to the total training loss so far and
			# calculate the number of correct predictions
			totalTrainLoss += loss
			trainCorrect += (torch.round(pred) == y).type(torch.float).mean().item()

		# Zeroing gradient, performing backpropagation, updating weights of the model
		# set the model in evaluation mode
		with torch.no_grad():
			model.eval() # prep model for evaluation
			# loop over the validation set
			for (x, y) in valloader:
				# send the input to the device
				(x, y) = (x.to(device), y.to(device))
				# make the predictions and calculate the validation loss
				pred = model(x)
				totalValLoss += lossFn(pred, y.type(torch.float32).unsqueeze(1))
				# calculate the number of correct predictions
				valCorrect += (torch.round(pred) == y).type(torch.float).mean().item()
	
		# calculate the average training and validation loss
		avgTrainLoss = totalTrainLoss / trainSteps
		avgValLoss = totalValLoss / valSteps
		
		# calculate the training and validation accuracy
		avgtrainCorrect = trainCorrect / len(trainloader)
		avgvalCorrect = valCorrect / len(valloader)
		
		# update our training history
		train_loss.append(avgTrainLoss.cpu().detach().numpy())
		train_acc.append(avgtrainCorrect)
		val_loss.append(avgValLoss.cpu().detach().numpy())
		val_acc.append(avgvalCorrect)
		
		# print the model training and validation information
		print("[INFO] Epoch: {}/{}".format(e + 1, num_epochs))
		print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(avgTrainLoss, avgtrainCorrect))
		print("Valid loss: {:.6f}, Valid accuracy: {:.4f}\n".format(avgValLoss, avgvalCorrect))

		# early_stopping needs the validation loss to check if it has decresed, 
		# and if it has, it will make a checkpoint of the current model
		early_stopping(avgValLoss, model)
        
		if early_stopping.early_stop:
			print("Early stopping")
			break
	
	# load the last checkpoint with the best model
	model.load_state_dict(torch.load('checkpoint.pt'))
	
	return  model, train_loss, train_acc, val_loss, val_acc

In [11]:
model, AvgTrainLoss, avgtrainCorrect, avgValLoss, avgvalCorrect = train_model(model, bs, patience, num_epochs)

[INFO] Epoch: 1/50
Train loss: 0.676057, Train accuracy: 0.4969
Valid loss: 0.569118, Valid accuracy: 0.7618

Validation loss decreased (inf --> 0.569118).  Saving model ...
[INFO] Epoch: 2/50
Train loss: 0.333889, Train accuracy: 0.5299
Valid loss: 0.266696, Valid accuracy: 0.8741

Validation loss decreased (0.569118 --> 0.266696).  Saving model ...
[INFO] Epoch: 3/50
Train loss: 0.175747, Train accuracy: 0.5342
Valid loss: 0.418893, Valid accuracy: 0.8017

EarlyStopping counter: 1 out of 10
[INFO] Epoch: 4/50
Train loss: 0.125097, Train accuracy: 0.5202
Valid loss: 0.174736, Valid accuracy: 0.9225

Validation loss decreased (0.266696 --> 0.174736).  Saving model ...
[INFO] Epoch: 5/50
Train loss: 0.110850, Train accuracy: 0.5251
Valid loss: 0.142808, Valid accuracy: 0.9328

Validation loss decreased (0.174736 --> 0.142808).  Saving model ...
[INFO] Epoch: 6/50
Train loss: 0.075110, Train accuracy: 0.5326
Valid loss: 0.084976, Valid accuracy: 0.9538

Validation loss decreased (0.14280

In [12]:
# visualize the loss as the network trained
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(AvgTrainLoss)+1),AvgTrainLoss, label='Training Loss')
plt.plot(range(1,len(avgValLoss)+1),avgValLoss,label='Validation Loss')

# find position of lowest validation loss
minposs = avgValLoss.index(min(avgValLoss))+1 
plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim(0, 0.5) # consistent scale
plt.xlim(0, len(AvgTrainLoss)+1) # consistent scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('loss_plot.png', bbox_inches='tight')

In [13]:
# finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

[INFO] total time taken to train the model: 5644.31s


In [29]:
def test_model(model, lossFn, testloader):
  model.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for (x, y) in testloader:
      out = model(x).flatten()
      test_loss += lossFn(out, y.type(torch.float32).unsqueeze(1))
      correct += torch.round(out).eq(y).sum()
      
    test_loss /= len(testloader)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(testloader),
            100. * correct / len(testloader)))
    
test_model(model, lossFn, test_data)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x676 and 10816x1024)

In [ ]:
# generate a classification report
print(classification_report(test_data.targets.numpy(),np.array(out), target_names=test_data.classes))